In [1]:
# path config
import os
dataroot = '/mmdetection3d/data/itri_dataset'
jsonfile = os.path.join(dataroot, 'label_3d/bevdet.json')

print('reading json file:')
print(jsonfile)

reading json file:
/mmdetection3d/data/itri_dataset/label_3d/bevdet.json


In [2]:
# read json
import json
from easydict import EasyDict as edict

with open(jsonfile, 'rb') as f:
    data = edict(json.load(f))

print('total frames:')
print(len(data.keys()))

total frames:
382


In [3]:
# define output
out = edict()
out.metadata = edict()
out.metadata.version = "itri-3d-all"
out.infos = []

print(f"""{len(out["infos"])}, {len(out["metadata"])}""")
print(out)

0, 1
{'metadata': {'version': 'itri-3d-all'}, 'infos': []}


In [4]:
itri_cam_config = {
    "image_front_bottom_60": {
        "cam_intrinsic": [[618.175335, 0.000000, 273.771329,], [0.000000, 611.31756, 204.595871,], [0.000000, 0.000000, 1.000000]],
        "sensor2lidar_rotation": [
            [0.9980077885482712, 0.0247124471717397, 0.05804953877312857],
            [0.0585477232287393, -0.01993650590000778, -0.9980855172966042],
            [0.0235078306468276, -0.9994957882288864, 0.02134364568196223]
        ],
        "sensor2lidar_translation": [-0.3237799271578172, 0.9653148462462031, -2.241561306526251],
    },
    "image_front_top_far_30": {
        "cam_intrinsic": [[1109.320305, 0.0000000, 336.718475], [0.000000, 1106.037227, 235.528447], [0.00000, 0.000000, 1.00000]],
        "sensor2lidar_rotation": [
            [0.9904617902131139, 0.01474944086427021, 0.1369959711890263],
            [0.1354022972149057, 0.08001517686519838, -0.9875544488179662],
            [0.02552763280759701, -0.9966845165173492, -0.0772548671464659]
        ],
        "sensor2lidar_translation": [0.1662200728421821, 1.165314846246203, -0.761561306526251],
    },
    "image_left_back_60": {
        "cam_intrinsic": [[618.175335, 0.000000, 273.771329,], [0.000000, 611.31756, 204.595871,], [0.000000, 0.000000, 1.000000]],
        "sensor2lidar_rotation": [
            [0.7574353003515645, 0.0486152312052206, -0.6510977845732533],
            [-0.6358769567282897, 0.2812238260917291, -0.7187305862006279],
            [-0.1481629565151313, -0.9584099952177003, -0.2439221584512227]
        ],
        "sensor2lidar_translation": [-6.713779927157818, 4.765314846246203, -2.871561306526251],
    },
    "image_right_back_60": {
        "cam_intrinsic": [[618.175335, 0.000000, 273.771329,], [0.000000, 611.31756, 204.595871,], [0.000000, 0.000000, 1.000000]],
        "sensor2lidar_rotation": [
            [0.8651667173805694, -0.01872477257324379, 0.5011346466060891],
            [0.4801479399561342, 0.3193161403620065, -0.817003768816394],
            [0.1447221713933642, -0.9474632369629923, -0.2852523579408208]
        ],
        "sensor2lidar_translation": [-12.42377992715782, -5.124685153753797, -3.261561306526251],
    },
}

In [5]:
frames = set(data.keys())
class_names = ['pedestrian', 'bicycle', 'car', 'motorbike', 'bus', 'truck', 'dontcare']

for frame in frames:
    i = edict()
    i.token = frame
    i.timestamp = frame
    i.num_lidar_pts = 1
    i.valid_flag = True
    i.gt_boxes = data[frame].array
    i.gt_labels_3d = data[frame]["class"]
    i.gt_names = [class_names[i] for i in data[frame]["class"]]

    i.ann_infos = (i.gt_boxes, i.gt_labels_3d)

    i.cams = edict()
    for cam, configs in itri_cam_config.items():
        cam_config = edict()
        for key, config in configs.items():
            cam_config[key] = config

        i.cams[cam] = cam_config
        i.cams[cam].type = cam
        i.cams[cam].data_path = os.path.join(dataroot, cam, f"{frame}.jpg")
    
    out.infos.append(i)

In [6]:
# export to pickle file
pickle_path = os.path.join(dataroot, "itri_infos_train.pkl")

with open(pickle_path, "wb") as f:
    import pickle
    pickle.dump(out, f)

pickle_path = os.path.join(dataroot, "itri_infos_val.pkl")

with open(pickle_path, "wb") as f:
    import pickle
    pickle.dump(out, f)

In [7]:
# examine exported pickle file
with open(pickle_path, "rb") as f:
    pkl = pickle.load(f)

print(pkl.infos[0].cams['image_front_bottom_60'].data_path)

print(pkl.infos[0].gt_boxes)
print(pkl.infos[0].gt_labels_3d)

/mmdetection3d/data/itri_dataset/image_front_bottom_60/1671585432.jpg
[[20.450435638427734, 0.6813616752624512, -2.4600727558135986, 1.96230947971344, 4.214895725250244, 1.6542085409164429, 0.277220547199249, 0, 0], [3.279998779296875, 12.560001373291016, -2.1618356704711914, 0.6244035959243774, 1.6121983528137207, 1.8127834796905518, 0.3638122677803034, 0, 0]]
[4, 3]
